# Point to point low-thrust transfer

In [1]:
import pykep as pk
import numpy as np
import time
import pygmo as pg
import pygmo_plugins_nonfree as ppnf

from matplotlib import pyplot as plt

In [225]:
class direct_point2point:
    """Represents a direct transcription of the optimal low-thrust transfer between two fixed points.

    This problem works using a Sims-Flanagan leg and manipulating its transfer time T, final mass mf and the controls.

    The decision vector is::

        z = [mf, throttles, tof]

    where throttles is a vector [u0x, u0y,u0z, ...]

    """

    def __init__(
        self,
        rvs=[
            np.array([1, 0.1, -0.1]) * pk.AU,
            np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY,
        ],
        rvf=[
            np.array([-1.2, -0.1, 0.1]) * pk.AU,
            np.array([0.2, -1.023, 0.44]) * pk.EARTH_VELOCITY,
        ],
        ms=1000,
        mu=pk.MU_SUN,
        max_thrust=0.12,
        isp=3000,
        tof_bounds=[80, 400],
        mf_bounds=[200.0, 1000.0],
        nseg=10,
        cut=0.6,
        mass_scaling = 1000
    ):
        # We add as data member one single Sims-Flanagan leg using the problem data
        # and some temporary (and unused, thus irrelelvant) values for the to-be-optimzed parameters throttles, tof and mf.
        throttles = np.random.uniform(-1, 1, size=(nseg * 3))
        self.leg = pk.leg.sims_flanagan(
            rvs=rvs,
            ms=ms,
            throttles=throttles,
            rvf=rvf,
            mf=np.mean(mf_bounds),
            tof=np.mean(tof_bounds) * pk.DAY2SEC,
            max_thrust=max_thrust,
            isp=isp,
            mu=mu,
            cut=cut,
            )
        self.tof_bounds = tof_bounds
        self.mf_bounds = mf_bounds
        self.mass_scaling = mass_scaling

    def get_bounds(self):
        lb = [self.mf_bounds[0]] + [-1, -1, -1] * self.leg.nseg + [self.tof_bounds[0]]
        ub = [self.mf_bounds[1]] + [1, 1, 1] * self.leg.nseg + [self.tof_bounds[1]]
        return (lb, ub)

    def _set_leg_from_x(self, x):
        # We set the leg using data in the decision vector
        self.leg.tof = x[-1] * pk.DAY2SEC
        self.leg.mf = x[0]
        self.leg.throttles = x[1:-1]

    def fitness(self, x):
        # 1 - We set the leg using data in the decision vector
        self._set_leg_from_x(x)
        obj = -x[0] / self.mass_scaling

        # 2 - We compute the constraints violations (mismatch+throttle)
        ceq = self.leg.compute_mismatch_constraints()
        cineq = self.leg.compute_throttle_constraints()
        retval = np.array([obj] + ceq + cineq)  # here we can sum lists

        # 3 - We scale the values in nd units (numerical solvers are sensitive to well-scaled values)
        retval[1:4] /= pk.AU
        retval[4:7] /= pk.EARTH_VELOCITY
        retval[7] /= self.mass_scaling

        return retval

    def gradientTMP(self, x):
        self._set_leg_from_x(x)
        mcg_xs, mcg_xf, mcg_th_tof = self.leg.compute_mc_grad()
        tcg_th = self.leg.compute_tc_grad()
        
        # The gradient of the objective function (obj = -mf)
        retval = [-1. / self.mass_scaling]
        # The gradient of the mismatch contraints (mcg). We divide them in pos, vel mass as they have different scaling units
        # pos
        for i in range(3):
            # First w.r.t. mf
            retval.append(mcg_xf[i,-1] / pk.AU)
            # Then the [throttles, tof]
            retval.extend(mcg_th_tof[i,:] / pk.AU)
            retval[-1] *= pk.DAY2SEC
        # vel
        for i in range(3,6):
            # First w.r.t. mf
            retval.append(mcg_xf[i,-1]/ pk.EARTH_VELOCITY)
            # Then the [throttles, tof]
            retval.extend(mcg_th_tof[i,:]/ pk.EARTH_VELOCITY)
            retval[-1] *= pk.DAY2SEC
        # mass
        for i in range(6,7):
            # First w.r.t. mf
            retval.append(mcg_xf[i,-1] / self.mass_scaling)
            # Then the [throttles, tof]
            retval.extend(mcg_th_tof[i,:]/self.mass_scaling)
            retval[-1] *= pk.DAY2SEC
        
        return retval
    
    def sparsity_pattern(self):
        dim = 2 + 3 * self.leg.nseg
        # The objective function only depends on the final mass, which is in the chromosome.
        retval = [[0,0]]
        # The mismatch constraints depend on all variables.
        for i in range(1,8):
            for j in range(dim):
                retval.append([i, j])
        # The throttle constraints only depend on the specific throttles (3).
        for i in range(self.leg.nseg):
            retval.append([8+i, 3*i+1])
            retval.append([8+i, 3*i+2])
            retval.append([8+i, 3*i+3])
        # We return the sparsity pattern
        return retval
        
    def get_nec(self):
        return 7

    def get_nic(self):
        return self.leg.nseg
    
udp = direct_point2point(
    rvs=[rs, vs],
    rvf=[rf, vf],
    mu=pk.MU_SUN,
    max_thrust=0.12,
    isp=3000,
    tof_bounds=[80, 200],
    mf_bounds=[200.0, 1000.0],
    nseg=nseg,
    cut=0.6,
)
x = pg.population(prob, 1).get_x()[0]



In [226]:
udp.gradientTMP(x)[1:15]

[-1.4694721245265518e-05,
 0.006813221805208961,
 0.00024430842703371714,
 4.925681352913288e-07,
 -0.01311342691525777,
 -0.0005014600058762055,
 -7.176711271919772e-07,
 -0.012056461006290039,
 1.0223476528838442e-05,
 0.0002459535545847036,
 0.0066233232757768315,
 3.6692385943127215e-07,
 -0.0004978984473423457,
 -0.013475900194643495]

In [227]:
pg.estimate_gradient_h(udp.fitness, x)[8:]

array([ 1.46947212e-05,  6.81322181e-03,  2.44308427e-04,  4.92568139e-07,
       -1.31134269e-02, -5.01460006e-04, -7.17671120e-07, -1.20564610e-02,
       -1.02234765e-05,  2.45953555e-04,  6.62332328e-03,  3.66923860e-07,
       -4.97898447e-04, -1.34759002e-02, -9.38387522e-07,  1.22893180e-02,
        4.30503734e-06,  5.01568579e-07,  3.70761390e-07,  6.45138920e-03,
       -7.25251065e-07, -9.56086319e-07, -1.27542482e-02,  2.88265433e-06,
       -3.48545582e-05,  1.75302490e-02,  1.88221314e-03,  4.60277248e-06,
        3.29808297e-02,  3.88879645e-03,  7.60535628e-06, -1.17575587e-02,
        2.33262850e-05,  1.90636474e-03,  1.66101347e-02,  3.87551494e-06,
        3.83641005e-03,  3.46299510e-02,  8.75315844e-06, -1.20734686e-02,
       -1.00550366e-05,  4.73490516e-06,  3.93185252e-06,  1.50889482e-02,
        7.71684839e-06,  9.01348745e-06,  2.97849419e-02,  9.27052114e-05,
       -9.99709368e-04, -1.50740597e-02,  2.55059386e-03, -5.64533104e-03,
       -1.35506791e-02,  

In [203]:
udp._set_leg_from_x(x)
mcg_xs, mcg_xf, mcg_th_tof = udp.leg.compute_mc_grad()
mcg_xf[1,-1]/pk.AU


0.00013955916376220278

In [207]:
# Problem data
mu = pk.MU_SUN
max_thrust = 0.12
isp = 3000

# Initial state
ms = 1500.0
rs = np.array([1, 0., -0.0]) * pk.AU
vs = np.array([0.01, 1, -0.0]) * pk.EARTH_VELOCITY

# Final state
mf = 1300.0
rf = np.array([-0.0, 1.0, 0.0]) * pk.AU
vf = np.array([-1, -0, 0]) * pk.EARTH_VELOCITY

# Throttles and tof
nseg = 2
throttles = np.random.uniform(-1, 1, size=(nseg * 3))
tof = 2 * np.pi * np.sqrt(pk.AU**3 / pk.MU_SUN) / 4

sf = pk.leg.sims_flanagan(
    rvs=[rs, vs],
    ms=ms,
    throttles=throttles,
    rvf=[rf, vf],
    mf=mf,
    tof=tof,
    max_thrust=max_thrust,
    isp=isp,
    mu=mu,
    cut=0.6,
)
udp = direct_point2point(
    rvs=[rs, vs],
    rvf=[rf, vf],
    mu=pk.MU_SUN,
    max_thrust=0.12,
    isp=3000,
    tof_bounds=[80, 200],
    mf_bounds=[200.0, 1000.0],
    nseg=nseg,
    cut=0.6,
)

In [209]:
snopt72 = "/usr/local/lib/libsnopt7_c.so"
uda = ppnf.snopt7(library=snopt72, minor_version=2, screen_output=True)
uda.set_integer_option("Major iterations limit", 700)
uda.set_integer_option("Iterations limit", 20000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-9)
algo = pg.algorithm(uda)

In [211]:
prob = pg.problem(udp)
pop = pg.population(prob, 1)
pop = algo.evolve(pop)

   SNOPT  C interface  2.0.0   
 S N O P T  7.2-4    (Jun 2006)
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
 

 Nonlinear constraints       9     Linear constraints       0
 Nonlinear variables         8     Linear variables         0
 Jacobian  variables         8     Objective variables      8
 Total constraints           9     Total variables          8
 

 
 The user has defined       0   out of      80   first  derivatives
 

 Major Minors     Step   nCon Feasible  Optimal  MeritFunction    nS Penalty
     0     10               1  3.0E-03  1.9E+00  4.4281307E+04                 r i
     1     12  7.3E-03      2  3.0E-03  6.1E-01  4.3958919E+04     2         n rli
     2      1  3.4E-01      4  2.9E-03  4.0E-01  4.0784665E+04     2         n rli
     3      4  1.0E+00      6  3.2E-03  3.8E-02  3.3622881E+04     3         n r i
     4     14  1.0E+00      8  2.1E-03  1.3E+00  2.7226988E+04     2         sm  i
     5     20  1.0E+00 

In [164]:
pop.get_f()[0]

array([-0.58381587,  0.13544511, -0.08880357, -0.00328173,  0.10637389,
        0.13912857,  0.00177073,  0.38522962,  0.44054429,  0.77396517,
       -0.13554324, -0.02249726, -0.25650703])

In [63]:
365.25/4

91.3125